<a href="https://colab.research.google.com/github/Abhipatel18/Chatbot/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tensorflow
import random
import json
import pickle
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
stemmer = LancasterStemmer()

# Load the intents file
with open("intents3.json") as file:
    data = json.load(file)

words = []
labels = []
docs_x = []
docs_y = []

# Tokenize each pattern in our dataset
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

# Reduce words to their root form and sort them
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

# Save words and labels
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(labels, open('labels.pkl', 'wb'))

In [9]:
# Creating bag-of-words encoding of the dataset
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

# Building the model
model = Sequential()
model.add(Dense(128, input_shape=(len(training[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(output[0]), activation='softmax'))

# Compile the model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [10]:
hist = model.fit(training, output, epochs=1000, batch_size=8, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/1000
159/159 [==============================] - 1s 3ms/step - loss: 5.2417 - accuracy: 0.0079
Epoch 2/1000
159/159 [==============================] - 0s 3ms/step - loss: 5.2078 - accuracy: 0.0213
Epoch 3/1000
159/159 [==============================] - 1s 3ms/step - loss: 5.1738 - accuracy: 0.0197
Epoch 4/1000
159/159 [==============================] - 0s 3ms/step - loss: 5.1373 - accuracy: 0.0276
Epoch 5/1000
159/159 [==============================] - 0s 3ms/step - loss: 5.1179 - accuracy: 0.0237
Epoch 6/1000
159/159 [==============================] - 0s 3ms/step - loss: 5.0864 - accuracy: 0.0245
Epoch 7/1000
159/159 [==============================] - 0s 3ms/step - loss: 5.0453 - accuracy: 0.0324
Epoch 8/1000
159/159 [==============================] - 0s 3ms/step - loss: 4.9968 - accuracy: 0.0434
Epoch 9/1000
159/159 [==============================] - 0s 3ms/step - loss: 4.9181 - accuracy: 0.0553
Epoch 10/1000
159/159 [==============================] - 0s 3ms/step - loss: 4.863

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents3.json').read())
words = pickle.load(open('words.pkl', 'rb'))
labels = pickle.load(open('labels.pkl', 'rb'))

In [12]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": labels[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    if not ints:
        return "I didn't understand that, can you please rephrase?"

    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [13]:
print("Start chatting with the bot (type 'quit' to stop)!")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    response = chatbot_response(message)
    print("Bot:", response)

Start chatting with the bot (type 'quit' to stop)!
You: hi
1/1 [==============================] - 0s 101ms/step
Bot: Hi there, how can I help?
You: your  name
1/1 [==============================] - 0s 23ms/step
Bot: I'm bot.
You: help me
1/1 [==============================] - 0s 23ms/step
Bot: tell me what can i do for you
You: flashbacks
1/1 [==============================] - 0s 22ms/step
Bot: you might be suffering from emotional problems
You: fever
1/1 [==============================] - 0s 23ms/step
Bot: I didn't understand that, can you please rephrase?
You: high temperature
1/1 [==============================] - 0s 20ms/step
Bot: Fever can be a sign of infection. Stay hydrated, rest, and consider taking fever-reducing medication like acetaminophen or ibuprofen. If the fever is very high or persists for more than a few days, see a healthcare provider.
You: blocked nose
1/1 [==============================] - 0s 39ms/step
Bot: Sinusitis can cause facial pain and a blocked nose. Stay 